In [1]:
library("readxl")

In [2]:
library("lubridate")


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [3]:
search()

[1] ".GlobalEnv"        "package:lubridate" "package:readxl"   
 [4] "jupyter:irkernel"  "package:stats"     "package:graphics" 
 [7] "package:grDevices" "package:utils"     "package:datasets" 
[10] "package:methods"   "Autoloads"         "package:base"

In [4]:
source("helpers.R")

In [5]:
path <- "../data/data.xlsx"

In [6]:
df <- DataFraming(path)

In [7]:
df <- df[-c(210),]

In [8]:
df$Date <- as.Date(df$Date)

In [9]:
firstYields <- df[df$Date == "2022-02-11",]

In [10]:
firstYields

,Date,10Y,7Y,5Y,3Y,2Y,1Y,6M,3M
209,2022-02-11,1.941,1.94,1.856,1.728,1.505,1.029,0.692,0.359


#### Suponemos que: 
- $\alpha_1$ representa la tasa forward entre 0 y 3 meses, 
- $\alpha_2$ la tasa entre 3 meses y 1 año, 
- $\alpha_3$ la tasa entre 1 y 3 años,
- $\alpha_4$ la tasa entre 3 y 5 años ,
- $\alpha_5$ la tasa entre 5 y 10 años. 

In [11]:
today <- firstYields$Date

#### Además, para traer a valor presente usando una curva forward, se sigue el factor de descuento a un plazo T es: 

### $$ e^{-\int_{0}^{T} f(t) \,dt}$$

#### Definamos algunos objetos de utilidad. Cuántos cupones paga cada bono? Cómo ajustamos la tasa para cada pago?

In [175]:
alpha <- c(0.1,0.11,0.12,0.13,0.14)

In [176]:
alpha

[1] 0.10 0.11 0.12 0.13 0.14

In [14]:
today = firstYields$Date

In [15]:
threeMonthBond <- 100 + RateConverter(firstYields[["3M"]],3,today)*DiscountFactor(alpha, 3)

In [16]:
sixMonthBond <- 100 + RateConverter(firstYields[["6M"]], 6, today)*DiscountFactor(alpha, 6)

In [17]:
sixMonthBond

[1] 100.1415

In [18]:
oneYearBond <- 100 +  RateConverter(firstYields[["1Y"]], 6, today) *DiscountFactor(alpha, 6) + RateConverter(firstYields[["1Y"]], 12, today)*DiscountFactor(alpha, 12)

In [19]:
oneYearBond

[1] 100.3381

In [20]:
maturity<- 36

In [21]:
periods <- seq(6, maturity, 6)

In [22]:
threeMonthBond

[1] 100.0657

In [23]:
singleBondPayment <- function(maturity, yield, alpha, today){
    return (RateConverter(yield, maturity,today)*DiscountFactor(alpha, maturity))
}

In [24]:
singleBondPayment(3,firstYields[["3M"]], alpha, today)

[1] 0.06574967

In [59]:
yieldList <- list(firstYields)

In [129]:
nombres <- colnames(firstYields)

In [130]:
nombres <- nombres[-1]

In [131]:
nombres

[1] "10Y" "7Y"  "5Y"  "3Y"  "2Y"  "1Y"  "6M"  "3M"

In [133]:
nombres <- as.list(nombres)

In [134]:
names(nombres) <- c(120,84,60,36,24,12,6,3) 

In [177]:
BondValue <- function (today, maturity, yieldlist,alpha ){
    if (maturity == 3){
        return (100 + singleBondPayment(3,yieldlist[["3M"]], alpha, today))
        }
    setter <- as.character(maturity)
    periods <- seq(6, maturity, 6)
    bondSum <- sum(unlist(lapply(periods, function(x) singleBondPayment(x, yield = yieldlist[[nombres[[setter]]]],  alpha = alpha, today = today))))
    return(bondSum +100)
}

In [189]:
BondValue(today, 120, firstYields, alpha)

[1] 102.0603

In [191]:
maturities <- c(120,84,60,36,24,12,6,3) 

In [192]:
bondValues <- list()
for (i in maturities){
    bondValues <- append(bondValues, BondValue(today, i, firstYields, alpha))
}

In [193]:
bondValues

[[1]]
[1] 102.0603

[[2]]
[1] 102.0589

[[3]]
[1] 101.9643

[[4]]
[1] 101.7449

[[5]]
[1] 101.3279

[[6]]
[1] 100.5627

[[7]]
[1] 100.1853

[[8]]
[1] 100.0657

In [207]:
error <-function(alpha){
    maturities <- c(3,6,12,24,36,60,84,120) 
    bondValues <- list()
    for (i in maturities){
        bondValues <- append(bondValues, BondValue(today, i, firstYields, alpha))
    }
    error <- sum((unlist(bondValues)-100)**2)
    return (error)
}

In [211]:
optim(alpha, error, lower = 0, upper= 1, method = "BFGS")

Warning message in optim(alpha, error, lower = 0, upper = 1, method = "BFGS"):
“bounds can only be used with method L-BFGS-B (or Brent)”

$par
[1] 1.0000000 1.0000000 1.0000000 1.0000000 0.6543727

$value
[1] 4.738658e-05

$counts
function gradient 
       2        2 

$convergence
[1] 0

$message
[1] "CONVERGENCE: NORM OF PROJECTED GRADIENT <= PGTOL"